## Apply the time seres analysis per each x, y location (pixel) within a single Equi7grid tile

In [1]:
import numpy as np
import os, osr, glob
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import pandas as pd
import xarray as xr
import rioxarray
import zarr
# import TUW packages
from yeoda.products.preprocessed import SIG0DataCube
from geopathfinder.naming_conventions.yeoda_naming import YeodaFilename
#
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import my aux functions
from auxilary_ts_tools_mm import plot_TS_fromPandaSeres, features_from_S1_TS, features_as_xrrray_ufunc

Specify the folders with a S1 datacube (a 300x300 m<sup>2</sup> Equi7Tile)

In [2]:
tile_dir1_path = r'/project/return/Share/EODC_SA020M/V01R01/E078N066T3'
tile_dir2_path = r'/project/return/Share/EODC_SA020M/V1M0R1/E078N066T3'
#tile_dir1_path = r'/project/return/Share/EODC_SA020M/V01R01/E078N060T3'
#tile_dir2_path = r'/project/return/Share/EODC_SA020M/V1M0R1/E078N060T3'
#tile_dir1_path = r'/project/return/Share/EODC_SA020M/V01R01/E078N060T3'
#tile_dir2_path = r'/project/return/Share/EODC_SA020M/V1M0R1/E051N060T3'
#tile_dir1_path = r'/project/return/Share/EODC_SA020M/V01R01/E051N060T3'
#tile_dir2_path = r'/project/return/Share/EODC_test1/E078N060T3'
# specify other parameters:
dimensions=['time', 'band', 'extra_field', 'sensor_field']
#
filepaths1 = glob.glob(os.path.join(tile_dir1_path,'*.tif'))
filepaths2 = glob.glob(os.path.join(tile_dir2_path,'*.tif'))

Get the lists of all tiles in the two folders with data and check if they are identical:

In [3]:
tile_names1 = [os.path.basename(aa) for aa in glob.glob(r'/project/return/Share/EODC_SA020M/V01R01/*')]
tile_names2 = [os.path.basename(aa) for aa in glob.glob(r'/project/return/Share/EODC_SA020M/V1M0R1/*')]

In [4]:
len(tile_names1)

106

In [40]:
text_file = open("equi7tile_list.txt", "r")
lines = text_file.read().split('\n')
print(lines)
print(len(lines))
text_file.close()

['E051N060T3', 'E051N063T3', 'E051N066T3', 'E051N069T3', 'E051N072T3', 'E054N057T3', 'E054N060T3', 'E054N063T3', 'E054N066T3', 'E054N069T3', 'E054N072T3', 'E054N075T3', 'E057N054T3', 'E057N057T3', 'E057N060T3', 'E057N063T3', 'E057N066T3', 'E057N069T3', 'E057N072T3', 'E057N075T3', 'E060N054T3', 'E060N057T3', 'E060N060T3', 'E060N063T3', 'E060N066T3', 'E060N069T3', 'E060N072T3', 'E060N075T3', 'E063N051T3', 'E063N054T3', 'E063N057T3', 'E063N060T3', 'E063N063T3', 'E063N066T3', 'E063N069T3', 'E063N072T3', 'E063N075T3', 'E063N078T3', 'E066N051T3', 'E066N054T3', 'E066N057T3', 'E066N060T3', 'E066N063T3', 'E066N066T3', 'E066N069T3', 'E066N072T3', 'E066N075T3', 'E066N078T3', 'E069N051T3', 'E069N054T3', 'E069N057T3', 'E069N060T3', 'E069N063T3', 'E069N066T3', 'E069N069T3', 'E069N072T3', 'E069N075T3', 'E069N078T3', 'E069N081T3', 'E072N051T3', 'E072N054T3', 'E072N057T3', 'E072N060T3', 'E072N063T3', 'E072N066T3', 'E072N069T3', 'E072N072T3', 'E072N075T3', 'E072N078T3', 'E072N081T3', 'E075N051T3', 'E075

In [30]:
len(tile_names2)

106

In [31]:
len(tile_names1)

106

In [32]:
len(lines)

106

The tile that is missing in EODC_SA020M/V1M0R1

In [33]:
tile_names1[np.where(~np.isin(tile_names1, tile_names2))[0][0]]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
YeodaFilename.fields_def

In [ ]:
dimensions=['time', 'band', 'extra_field', 'sensor_field', 'data_version']

Read the datacube:

In [ ]:
sig0_dc1 = SIG0DataCube(filepaths=filepaths1, dimensions=dimensions, filename_class=YeodaFilename, sres=20, continent='SA')
sig0_dc2 = SIG0DataCube(filepaths=filepaths2, dimensions=dimensions, filename_class=YeodaFilename, sres=20, continent='SA')
# get info:
sig0_dc2.inventory[dimensions].head(5)

Filter by date:

In [ ]:
#toi_start, toi_end = datetime(2017, 1, 1), datetime(2023, 1, 1)
toi_start, toi_end = datetime(2017, 1, 1), datetime(2021, 1, 1)
sig0_dc1 = sig0_dc1.filter_by_dimension([(toi_start, toi_end)], [(">=", "<")], name="time", inplace=True)
sig0_dc2 = sig0_dc2.filter_by_dimension([(toi_start, toi_end)], [(">=", "<")], name="time", inplace=True)

Select bands:

In [ ]:
sig0_vv_dc1 = sig0_dc1.filter_by_dimension('VV', name='band')
sig0_vh_dc1 = sig0_dc1.filter_by_dimension('VH', name='band')
#
sig0_vv_dc2 = sig0_dc2.filter_by_dimension('VV', name='band')
sig0_vh_dc2 = sig0_dc2.filter_by_dimension('VH', name='band')

Merge and sort the datacubes:

In [ ]:
sig0_vv_dc = sig0_vv_dc1.unite(sig0_vv_dc2)
sig0_vv_dc = sig0_vv_dc.sort_by_dimension('time', ascending=True)
#
sig0_vh_dc = sig0_vh_dc1.unite(sig0_vh_dc2)
sig0_vh_dc = sig0_vh_dc.sort_by_dimension('time', ascending=True)
#
sig0_vv_dc.inventory

In [ ]:
# get a path of a class function
import inspect
print(inspect.getfile(sig0_vv_dc.filter_by_dimension))

In [ ]:
# filter all descendig images:
#
# get the unique list of descending orbits:
desc_list = [aa for aa in sig0_vv_dc.inventory.extra_field.unique().tolist() if aa[0]=='D']
asce_list = [aa for aa in sig0_vv_dc.inventory.extra_field.unique().tolist() if aa[0]=='A']
# filter
sig0_vv_desc_dc = sig0_vv_dc.filter_by_dimension(desc_list, name="extra_field")
sig0_vh_desc_dc = sig0_vh_dc.filter_by_dimension(desc_list, name="extra_field")
#
sig0_vv_asce_dc = sig0_vv_dc.filter_by_dimension(asce_list, name="extra_field")
sig0_vh_asce_dc = sig0_vh_dc.filter_by_dimension(asce_list, name="extra_field")
#
sig0_vh_desc_dc.inventory

In [ ]:
# filter by orbit

In [1]:
desc_list

NameError: name 'desc_list' is not defined

In [ ]:

sig0_vh_desc_dc_orbit1 = sig0_vh_desc_dc.filter_by_dimension('D142', name='extra_field')
sig0_vv_desc_dc_orbit1 = sig0_vv_desc_dc.filter_by_dimension('D142', name='extra_field')
#
sig0_vh_desc_dc_orbit1.inventory

In [ ]:
sig0_vv_asce_dc_orbit1 = sig0_vv_asce_dc.filter_by_dimension('A091', name='extra_field')
sig0_vv_asce_dc_orbit1.inventory

Specify indexing for looping trough individual chunks within the Equi7grid tile:

In [ ]:
# read a raster to get the size of x and y coordinates:
#single_tif_ds = xr.open_dataset(sig0_vv_dc.inventory.filepath[20], engine="rasterio") 
single_tif_ds = xr.open_dataset(sig0_vv_dc.inventory.filepath[20], engine="rasterio") 
#
#my_file_path = r'/project/return/Share/EODC_SA020M/V1M0R1/E078N060T3/SIG0_20180807T091643__VV_D141_E078N060T3_SA020M_V1M0R1_S1AIWGRDH.tif'
#
#single_tif_ds = xr.open_dataset(my_file_path, engine="rasterio") 

In [ ]:
single_tif_ds

In [ ]:
my_chunk_size = 1000
#
steps_row = np.arange(0, len(single_tif_ds.x)/my_chunk_size).astype(int)
steps_col = np.arange(0, len(single_tif_ds.y)/my_chunk_size).astype(int)

In [ ]:
steps_row

Load data 

In [ ]:
sig0_vv_dc_chunk1 = sig0_vv_desc_dc_orbit1.load_by_pixels(12*1000, 0*1000, row_size=10, col_size=10, dtype='xarray')
sig0_vh_dc_chunk1 = sig0_vh_desc_dc_orbit1.load_by_pixels(12*1000, 0*1000, row_size=10, col_size=10, dtype='xarray')
#sig0_vv_dc_chunk1 = sig0_vv_desc_dc.load_by_pixels(12*1000, 0*1000, row_size=10, col_size=10, dtype='xarray')
#sig0_vh_dc_chunk1 = sig0_vh_desc_dc.load_by_pixels(12*1000, 0*1000, row_size=10, col_size=10, dtype='xarray')

Get info

In [ ]:
print(sig0_vv_dc_chunk1)

Get the in-memory size 

In [ ]:
print('Datacube size in memory is:', np.round(sig0_vv_dc_chunk1.nbytes/(1024*1024), 1), 'MB')

Rename the variavle

In [ ]:
sig0_vv_dc_chunk1 = sig0_vv_dc_chunk1.rename({'1':'sig0_vv'})

In [ ]:
sig0_vh_dc_chunk1 = sig0_vh_dc_chunk1.rename({'1':'sig0_vh'})

Plot a time seres and inspect values:

In [ ]:
sig0_vv_dc_chunk1.sig0_vv.isel(x=0, y=0).dropna(dim='time').plot(linestyle='-', color='r')

In [ ]:
sig0_vh_dc_chunk1.sig0_vh.isel(x=0, y=0).dropna(dim='time').plot(linestyle='-', color='r')

Rescale the data in 2019 and 2020

In [ ]:
sig0_vv_dc_chunk1['sig0_vv'].loc[slice('2019-1-1','2021-1-1'), :, :] = sig0_vv_dc_chunk1.sel(time=slice('2019-1-1','2021-1-1')).apply(lambda x: np.round(x/10.,1)).sig0_vv.values
sig0_vh_dc_chunk1['sig0_vh'].loc[slice('2019-1-1','2021-1-1'), :, :] = sig0_vh_dc_chunk1.sel(time=slice('2019-1-1','2021-1-1')).apply(lambda x: np.round(x/10.,1)).sig0_vh.values

In [ ]:
sig0_vv_dc_chunk1.sig0_vv.isel(x=0, y=0).dropna(dim='time').plot(linestyle='-', color='r')

In [ ]:
sig0_vh_dc_chunk1.sig0_vh.isel(x=0, y=0).dropna(dim='time').plot(linestyle='-', color='r')

 Plot a single Sigmma0 image 

In [ ]:
#%matplotlib widget
%matplotlib inline
#sig0_vv_dc_chunk1.sig0_vv.isel(time=-4).plot(cmap='Greys_r', vmin=-10, vmax=-5)
#sig0_vh_dc_chunk1.sig0_vh.isel(time=-4).plot(cmap='Greys_r',vmin=-15, vmax=-5)
sig0_vh_dc_chunk1.sig0_vh.mean(dim='time').plot(cmap='Greys_r', vmin=-30, vmax=-20)

Convert to pandas seres and plot

In [ ]:
#sig0_vv_ts = sig0_vv_dc_chunk1.sig0_vv.isel(x=0, y=0).dropna(dim='time').to_series()
sig0_vv_ts = sig0_vv_dc_chunk1.sig0_vv.sel(x=7804427, y=6644722, method="nearest").dropna(dim='time').to_series()

In [ ]:
my_xticks = pd.date_range(datetime(2017,1,1), datetime(2021,1,1), freq='YS')
sig0_vv_ts.plot(style='ro-', xticks=my_xticks, grid=True, figsize=(14,4), legend=True, xlabel='Time', ylabel='Bacscatter Intensity [dB]')

Round the time and resample to 6 day TS:

In [ ]:
sig0_vv_ts.index = sig0_vv_ts.index.round('D')
#
sig0_vv_ts_6d = sig0_vv_ts.resample('6D').interpolate(method='linear')

In [ ]:
sig0_vv_ts_6d.index.year

In [ ]:
sig0_vv_ts_6d.groupby(sig0_vv_ts_6d.index.year).quantile(0.9).loc[2017]

In [ ]:
sig0_vv_ts_6d.min()

In [ ]:
sig0_vv_ts_6d.plot(style='bo-',grid=True, figsize=(14,4), legend=True, xlabel='Time', ylabel='Bacscatter Intensity [dB]')

In [ ]:
#%matplotlib widget
%matplotlib inline
plot_TS_fromPandaSeres(sig0_vv_ts_6d)

In [ ]:
myFeatures_v2 = features_from_S1_TS(sig0_vv_ts_6d)
myFeatures_v2

## xarray-approach: apply the time-seres analysis per each x, y location in xarray

Prepare timestamps

In [ ]:
ts_time_stamps = sig0_vv_dc_chunk1['sig0_vv'][:,0, 0].time.values

Get features per each pixel

In [ ]:
#%%timeit
dist_out = xr.apply_ufunc(features_as_xrrray_ufunc, 
                          sig0_vh_dc_chunk1['sig0_vh'],
                          ts_time_stamps,
                          input_core_dims=[["time"], []],
                          output_core_dims=[["features"]]
                         )

In [ ]:
#%%timeit
dist_out = xr.apply_ufunc(features_as_xrrray_ufunc, 
                          sig0_vv_dc_chunk1['sig0_vv'],
                          ts_time_stamps,
                          input_core_dims=[["time"], []],
                          output_core_dims=[["features"]]
                         )

In [ ]:
# timeit, 200x200 pixels: 4min 49s ± 4.23 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# timeit, 100x100 pixels: 1min 13s ± 451 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# timeit, 50x50 pixel: 18.6 s ± 91.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# timeit, 10x10 pixels: 741 ms ± 2.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# convert output to dataset
dist_out_ds = dist_out.to_dataset(dim='features')

In [ ]:
dist_out_ds = dist_out_ds.rename({0:'exception_label', 1:'ref_mean', 2:'error_margin',
                                  3:'num_of_segments', 4:'TS_end_flag', 5:'TS_end_flag_long', 6:'TS_end_mag',
                                  7:'seg_id', 8:'seg_size', 9:'max_mag', 10:'max_mag_date', 11:'t_pre', 12:'t_post', 13:'t_total',
                                  14:'max_mag_org', 15:'max_mag_org_date', 16:'t_mag_org',
                                  17:'seg2_size'})

In [ ]:
dist_out_ds

In [ ]:
print('Datacube size in memory is:', np.round(dist_out_ds.nbytes/(1024*1024), 1), 'MB')

In [ ]:
#d plotting:
#dist_out_ds.seg_size.astype('float').plot()
feature_da = dist_out_ds.exception_label.astype('int')
feature_da.plot(figsize=(10, 8))

get a list of row and columnns where the exception label is 0

In [ ]:
aa = np.column_stack(np.where(feature_da.where(feature_da == 0).values))

In [ ]:
aa[20000, :]

### Export

In [ ]:
type(dist_out_ds)

In [ ]:
dist_out_ds.to_zarr(os.path.join(r'/home/return-mmilenkovic/', 'test_chunk_export_vh.zarr'), 'w')

In [ ]:
dist_out_ds.to_netcdf(os.path.join(r'/home/return-mmilenkovic/', 'test_chunk_export_vh.nc'))

## Check the output

In [ ]:
#my_out = xr.open_dataset("/home/return-mmilenkovic/E078N066T3_1_1_100_VH.nc")
#my_out = xr.open_dataset("/project/return/Share/mm/S1_SA_output/E078N066T3_10_7_1000_VH.nc")
my_out = xr.open_dataset("E078N066T3_14_14_1000_VH.nc")

In [ ]:
my_out

In [ ]:
feature_da = my_out.max_mag.astype('float64')
feature_da.plot(figsize=(10, 8))

In [ ]:
np.nanmedian(my_out.max_mag.values.flatten())